In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd
import mlflow
import os
import numpy as np

In [4]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("arboles")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1713174288421, experiment_id='1', last_update_time=1713174288421, lifecycle_stage='active', name='arboles', tags={}>

# Evaluación Por Pos Sobre General

In [58]:
df_original = df.copy()
# Crear y entrenar el modelo de árbol de decisión
clf = DecisionTreeRegressor(criterion="absolute_error", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado
y_pred_rf = clf.predict(X_test)

# Calcular el error
error = mean_absolute_error(y_test, y_pred_rf)
print("Arbol de decision:", error)

# Crear DataFrame de predicciones vs. reales
pred_vs_real = pd.DataFrame({'y_test': y_test, 'y_pred_rf': y_pred_rf, 'dif': abs(y_test-y_pred_rf)})
posiciones = ['Left-Back', 'Centre-Back', 'Defensive Midfield',
       'Centre-Forward','Second Striker','Central Midfield', 'Left Winger',
       'Attacking Midfield', 'Right-Back','Right Winger',
       'Left Midfield', 'Goalkeeper']
# Mostrar el DataFrame
print(pred_vs_real)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="COMPORTAMIENTO POSICIONES EN MODELO COMPLETO") as run:
    # Registrar los hiperparámetros
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    
    # Registrar la métrica de error
    mlflow.log_metric("mean_absolute_error", error)
    
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar el DataFrame como un artefacto
    pred_vs_real.to_csv("pred_vs_real.csv", index=False)
    mlflow.log_artifact("pred_vs_real.csv")
    
    # Calcular medias por grupos de posiciones
    li = []
    for p in posiciones:
        solo_posicion = df_original.loc[pred_vs_real.index][df_original.loc[pred_vs_real.index, 'position_' + p] == 1]
        mean_dif = np.mean(pred_vs_real.loc[solo_posicion.index]['dif'])
        li.append(mean_dif)

    mean_defenders = np.mean(li[0:3])
    mean_strikers = np.mean(li[3:5])
    mean_midfielders = np.mean(li[5:11])
    mean_goalkeepers = li[11]

    print('Defensas', mean_defenders)
    print('Delanteros', mean_strikers)
    print('Medios', mean_midfielders)
    print('Porteros', mean_goalkeepers)

    # Guardar las métricas específicas en MLflow
    mlflow.log_metric("mean_error_defenders", mean_defenders)
    mlflow.log_metric("mean_error_strikers", mean_strikers)
    mlflow.log_metric("mean_error_midfielders", mean_midfielders)
    mlflow.log_metric("mean_error_goalkeepers", mean_goalkeepers)

Arbol de decision: 3293565.0887573967
          y_test   y_pred_rf        dif
893    1000000.0   1000000.0        0.0
827    1200000.0   1200000.0        0.0
149    8000000.0   6000000.0  2000000.0
872    3000000.0   1200000.0  1800000.0
1173  14000000.0  10000000.0  4000000.0
...          ...         ...        ...
692     150000.0    200000.0    50000.0
2091   1500000.0    800000.0   700000.0
1103   6000000.0   2500000.0  3500000.0
1589   4000000.0   9500000.0  5500000.0
605    5000000.0   3000000.0  2000000.0

[676 rows x 3 columns]


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Defensas 3308995.7264957265
Delanteros 1264224.1379310344
Medios 4033686.1604711376
Porteros 2029032.2580645161


# Con Hiperparámetros

In [5]:
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import preprocesamiento_datos
RANDOM_STATE = 83
X_train, X_val, X_test, y_train, y_val, y_test = preprocesamiento_datos.preprocesamiento(True,[], False)
clf = DecisionTreeRegressor(criterion="absolute_error", max_depth=30, max_features=None, max_leaf_nodes=80, min_samples_leaf=5, min_samples_split=12, splitter="random", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado en el conjunto de validación
y_pred_val = clf.predict(X_val)

# Predecir con el modelo entrenado en el conjunto de prueba
y_pred_test = clf.predict(X_test)

# Calcular el error en el conjunto de validación
error_val = mean_absolute_error(y_val, y_pred_val)
print("Árbol de decisión - Validación:", error_val)

# Calcular el error en el conjunto de prueba
error_test = mean_absolute_error(y_test, y_pred_test)
print("Árbol de decisión - Prueba:", error_test)

# Crear DataFrame de predicciones vs. reales para el conjunto de validación
pred_vs_real_val = pd.DataFrame({'y_val': y_val, 'y_pred_val': y_pred_val, 'dif_val': abs(y_val - y_pred_val)})

# Crear DataFrame de predicciones vs. reales para el conjunto de prueba
pred_vs_real_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test, 'dif_test': abs(y_test - y_pred_test)})

# Mostrar los DataFrames
print("Predicciones vs. reales - Validación:")
print(pred_vs_real_val)

print("Predicciones vs. reales - Prueba:")
print(pred_vs_real_test)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="FINAL MODEL TREE") as run:
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("max_features", None)
    mlflow.log_param("max_leaf_nodes", 80)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("min_samples_split", 12)
    mlflow.log_param("splitter", "random")
    # Registrar la métrica de error en el conjunto de validación
    mlflow.log_metric("mean_absolute_error_val", error_val)
    # Registrar la métrica de error en el conjunto de prueba
    mlflow.log_metric("mean_absolute_error_test", error_test)
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar los DataFrames como un artefacto
    pred_vs_real_val.to_csv("pred_vs_real_val.csv", index=False)
    mlflow.log_artifact("pred_vs_real_val.csv")
    
    pred_vs_real_test.to_csv("pred_vs_real_test.csv", index=False)
    mlflow.log_artifact("pred_vs_real_test.csv")


Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Árbol de decisión - Validación: 3325382.003395586
Árbol de decisión - Prueba: 3076823.84341637
R^2 ajustado - Validación: 0.6974377642330392
R^2 ajustado - Prueba: 0.6403587863414837
Predicciones vs. reales - Validación:
           y_val  y_pred_val     dif_val
1100    200000.0    300000.0    100000.0
1101  30000000.0  25000000.0   5000000.0
1102  30000000.0  28000000.0   2000000.0
1103   6000000.0   3200000.0   2800000.0
1104  40000000.0  24750000.0  15250000.0
...          ...         ...         ...
1684   1000000.0    300000.0    700000.0
1685   1000000.0    300000.0    700000.0
1686    800000.0    800000.0         0.0
1687    500000.0    300000.0    200000.0
1688    500000.0   1000000.0    500000.0

[589 rows x 3 columns]
Predicciones vs. reales - Prueba:
          y_test  y_pred_test    dif_test
1689  25000000.0   32000000.0   7000000.0
1690  25000000.0 

# Sin hiperparámetros

In [6]:
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import preprocesamiento_datos
RANDOM_STATE = 83
X_train, X_val, X_test, y_train, y_val, y_test = preprocesamiento_datos.preprocesamiento(True,[], False)
clf = DecisionTreeRegressor(criterion="absolute_error", random_state=RANDOM_STATE)
clf.fit(X_train, y_train)

# Predecir con el modelo entrenado en el conjunto de validación
y_pred_val = clf.predict(X_val)

# Predecir con el modelo entrenado en el conjunto de prueba
y_pred_test = clf.predict(X_test)

# Calcular el error en el conjunto de validación
error_val = mean_absolute_error(y_val, y_pred_val)
print("Árbol de decisión - Validación:", error_val)

# Calcular el error en el conjunto de prueba
error_test = mean_absolute_error(y_test, y_pred_test)
print("Árbol de decisión - Prueba:", error_test)

# Crear DataFrame de predicciones vs. reales para el conjunto de validación
pred_vs_real_val = pd.DataFrame({'y_val': y_val, 'y_pred_val': y_pred_val, 'dif_val': abs(y_val - y_pred_val)})

# Crear DataFrame de predicciones vs. reales para el conjunto de prueba
pred_vs_real_test = pd.DataFrame({'y_test': y_test, 'y_pred_test': y_pred_test, 'dif_test': abs(y_test - y_pred_test)})

# Mostrar los DataFrames
print("Predicciones vs. reales - Validación:")
print(pred_vs_real_val)

print("Predicciones vs. reales - Prueba:")
print(pred_vs_real_test)

# Iniciar una nueva run de MLflow
with mlflow.start_run(run_name="MODEL Sin HiperParámetros") as run:
    mlflow.log_param("criterion", "mae")
    mlflow.log_param("random_state", RANDOM_STATE)
    mlflow.log_param("max_depth", 30)
    mlflow.log_param("max_features", None)
    mlflow.log_param("max_leaf_nodes", 80)
    mlflow.log_param("min_samples_leaf", 5)
    mlflow.log_param("min_samples_split", 12)
    mlflow.log_param("splitter", "random")
    # Registrar la métrica de error en el conjunto de validación
    mlflow.log_metric("mean_absolute_error_val", error_val)
    # Registrar la métrica de error en el conjunto de prueba
    mlflow.log_metric("mean_absolute_error_test", error_test)
    # Guardar el modelo
    mlflow.sklearn.log_model(clf, "decision_tree_model")
    
    # Guardar los DataFrames como un artefacto
    pred_vs_real_val.to_csv("pred_vs_real_val.csv", index=False)
    mlflow.log_artifact("pred_vs_real_val.csv")
    
    pred_vs_real_test.to_csv("pred_vs_real_test.csv", index=False)
    mlflow.log_artifact("pred_vs_real_test.csv")

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Árbol de decisión - Validación: 3664983.0220713075
Árbol de decisión - Prueba: 3806628.1138790036
Predicciones vs. reales - Validación:
           y_val  y_pred_val     dif_val
1100    200000.0    600000.0    400000.0
1101  30000000.0  25000000.0   5000000.0
1102  30000000.0  25000000.0   5000000.0
1103   6000000.0  20000000.0  14000000.0
1104  40000000.0  28000000.0  12000000.0
...          ...         ...         ...
1684   1000000.0   5000000.0   4000000.0
1685   1000000.0    150000.0    850000.0
1686    800000.0    400000.0    400000.0
1687    500000.0    600000.0    100000.0
1688    500000.0    600000.0    100000.0

[589 rows x 3 columns]
Predicciones vs. reales - Prueba:
          y_test  y_pred_test    dif_test
1689  25000000.0   32000000.0   7000000.0
1690  25000000.0   24000000.0   1000000.0
1691  30000000.0   40000000.0  10000000.0
1692    300000.0  

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
